In [1]:
import pandas as pd
import numpy as np
import json

# Preguntas Semana de la ciencia

Estos datos corresponden a parejas de estudiantes. Sin embargo, en este dataset se tratará a cada pareja como un único usuario. Las respuestas están almacenadas en un fichero JSON que se encuentra en la carpeta `/data/MNCN`. 

Cada usuario ha respondido a las siguientes preguntas:

**103** - Consumo responsable

*Un consumo responsable de nuestros recursos ayuda a reducir la pérdida de biodiversidad, pero… ¿estamos dispuestos a hacer cambios en nuestro día a día?*
*¿En qué estarías dispuesto a hacer un consumo más responsable? (elige máximo 3)*

0. Reducir el tiempo de la ducha
1. Comprar menos ropa
2. No usar productos con muchos envases
3. Ir a pie a más sitios
4. Reducir los residuos que genero
5. Reciclar de forma correcta

**107** - Emisiones de CO2 y medios de transporte

**A** - *Una de las causas del cambio climático es las emisiones de CO2.*
*Para reducir estas emisiones podemos movernos de manera sostenible, yendo a los sitios andando, utilizando bicicletas o patinetes o utilizando el transporte público.*
*¿Cómo vas normalmente al colegio?*

0. En coche
1. Andando
2. En bicicleta
3. En autobús
4. En metro
5. En patinete


**B** - *¿Estarías dispuesto a cambiar el transporte e ir al colegio de una forma más sostenible?*

0. Sí
1. No
2. Quizá

**110** - *Quiz: Especies invasoras*

**A** - *Las especies invasoras acuáticas eliminan a las autóctonas generando pérdida de biodiversidad.*
*¿Has tenido alguna especie exótica como mascota? (cotorra argentinas, reptiles, aves, etc)*

0. Sí
1. No

**B** - *Si pudieras adoptar a un animal, ¿cuál escogerías?*

0. Tortuga de Florida
1. Perro común
2. Gato común
3. Cotorra Argentina
4. Cacatúa
5. Mono Capuchino




In [2]:
tests = [
    {
        'name': 'Colegio Arcangel',
        'grade': '5 - Primaria',
        'file': 'sesion_miercoles_11.json',
        'type': 'Privado',
        'zone': 'Extrarradio'
    },
    {
        'name': 'Colegio Arcangel',
        'grade': '6 - Primaria',
        'file': 'sesion_miercoles_12.json',
        'type': 'Privado',
        'zone': 'Extrarradio'
    },
    {
        'name': 'Colegio Nuestra Señora del Val',
        'grade': '6 - Primaria',
        'file': 'sesion_jueves_11.json',
        'type': 'Público',
        'zone': 'Extrarradio'
    },
    {
        'name': 'Colegio Nuestra Señora del Val',
        'grade': '6 - Primaria',
        'file': 'sesion_jueves_12.json',
        'type': 'Público',
        'zone': 'Extrarradio'
    },
    {
        'name': 'Colegio San francisco de Asis',
        'grade': '2 - ESO',
        'file': 'sesion_viernes_11.json',
        'type': 'Privado',
        'zone': 'Centro'
    },
    {
        'name': 'CEPA Aluche',
        'grade': 'Adultos',
        'file': 'sesion_viernes_12.json',
        'type': 'Público',
        'zone': 'Centro'
    }
]

In [3]:
# Cabecera dataset
head = ['UserId', 'School', 'Grade', 'Type', 'Zone', 'Timestamp', 'q103', 'q107-A', 'q107-B', 'q110-A', 'q110-B']
data = []


In [4]:
PATH_DATA = '../../data/MNCN/'
rows = []

for t in tests:
    f = open(PATH_DATA + t['file'])

    data = json.load(f)
    

    for d in data:
        row = []
        row.append(d['id'])
        row.append(t['name'])
        row.append(t['grade'])
        row.append(t['type'])
        row.append(t['zone'])
        row.append(d['serverTimestamp'])
        row.append(d['value']['selectManyQuizesAnswers'][0])

        if len(d['value']['selectManyQuizesAnswers'][1]) > 0:
            row.append(d['value']['selectManyQuizesAnswers'][1][0])
        else:
            row.append('NaN')

        if len(d['value']['selectManyQuizesAnswers'][2]) > 0:   
            row.append(d['value']['selectManyQuizesAnswers'][2][0])
        else:
            row.append('NaN')

        if len(d['value']['selectManyQuizesAnswers'][3]) > 0:
            row.append(d['value']['selectManyQuizesAnswers'][3][0])
        else:
            row.append('NaN')

        if len(d['value']['selectManyQuizesAnswers'][4]) > 0:
            row.append(d['value']['selectManyQuizesAnswers'][4][0])
        else:
            row.append('NaN')

        rows.append(row)

In [5]:
mncn_df = pd.DataFrame(rows, columns=head)

In [6]:
# Limpiamos el dataset
mncn_df['q103'] = mncn_df['q103'].apply(lambda row: ';'.join(map(str, np.sort(row))))
mncn_df = mncn_df[~(mncn_df['q107-A'] == 'NaN')]

In [7]:
# Guardamos el dataset
mncn_df.to_csv('../../data/MNCN/semana_ciencia_2021.csv', index=False)

In [8]:
# Preparamos los datos que usaremos en el clustering
# El objeto que pasamos es un array de 1 y 0s dependiendo de las respuestas qua han contestado
# Ejemplo [0, 1, 1, 0, 1, 0, ...]

# Primera posición de las respuestas de cada pregunta
init_pos_q103 = 0
init_pos_q107a = 6
init_pos_q107b = 12
init_pos_q110a = 15
init_pos_q110b = 17
len_q = 23

In [9]:
def get_responses_profile(row):
    result = [row['UserId']]
    prof = np.zeros(len_q)
    
    if row['q103'] != '':
        if ';' in row['q103']:
        
            for r in row['q103'].split(';'):
                prof[init_pos_q103 + int(r)] = 1

        else:
            prof[init_pos_q103 + int(row['q103'])] = 1

    prof[init_pos_q107a + row['q107-A']] = 1
    prof[init_pos_q107b + row['q107-B']] = 1
    prof[init_pos_q110a + row['q110-A']] = 1
    prof[init_pos_q110b + row['q110-B']] = 1

    for p in prof:
        result.append(p)

    return pd.Series(result)

In [10]:
profiles = mncn_df.apply(lambda row: get_responses_profile(row), axis=1)

In [11]:
columns = ['UserId']
columns.extend(['q103_' + str(i) for i in range(init_pos_q107a)])
columns.extend(['q107a_' + str(i) for i in range(6)])
columns.extend(['q107b_' + str(i) for i in range(3)])
columns.extend(['q110a_' + str(i) for i in range(2)])
columns.extend(['q110b_' + str(i) for i in range(6)])
profiles.columns = columns

In [12]:
mncn_profiles_df = pd.DataFrame(profiles)

In [13]:
mncn_df[['UserId', 'School', 'Grade', 'Type', 'Zone']]

,UserId,School,Grade,Type,Zone
0,3bcd671c-771d-49f3-a753-f8e5c8d2c00a,Colegio Arcangel,5 - Primaria,Privado,Extrarradio
1,b836965f-0f1f-4e60-9adf-63f7d9ced35b,Colegio Arcangel,5 - Primaria,Privado,Extrarradio
2,6753a984-128e-489e-9d49-c5406ccef6e6,Colegio Arcangel,5 - Primaria,Privado,Extrarradio
3,3fee2686-e5f9-481a-8cc3-5add84863429,Colegio Arcangel,5 - Primaria,Privado,Extrarradio
4,bc21b0cf-28d0-4bf8-8c44-a3f55ac331ef,Colegio Arcangel,5 - Primaria,Privado,Extrarradio
...,...,...,...,...,...
58,13d21c1f-e812-4143-a08e-d9addcf7e147,CEPA Aluche,Adultos,Público,Centro
59,96f9ae20-9cb1-40b1-a550-c86d6b9c4ef5,CEPA Aluche,Adultos,Público,Centro
60,2d3f127b-0d5d-4f3e-b656-11a1ecd7e67d,CEPA Aluche,Adultos,Público,Centro
61,9e7633aa-857c-417f-839b-c3e0995144ac,CEPA Aluche,Adultos,Público,Centro


In [14]:
mncn_profiles_df = mncn_profiles_df.set_index('UserId').join(mncn_df[['UserId', 'School', 'Grade', 'Type', 'Zone']].set_index('UserId'), on='UserId')

In [15]:
mncn_profiles_df.to_csv('../../data/MNCN/user_profiles.csv')

In [16]:
mncn_profiles_df.columns

Index(['q103_0', 'q103_1', 'q103_2', 'q103_3', 'q103_4', 'q103_5', 'q107a_0',
       'q107a_1', 'q107a_2', 'q107a_3', 'q107a_4', 'q107a_5', 'q107b_0',
       'q107b_1', 'q107b_2', 'q110a_0', 'q110a_1', 'q110b_0', 'q110b_1',
       'q110b_2', 'q110b_3', 'q110b_4', 'q110b_5', 'School', 'Grade', 'Type',
       'Zone'],
      dtype='object')

## Agrupar respuestas

A continuación, generaremos un segundo dataset donde se agruparán las respuestas según el tipo.

In [17]:
answers_q103 = {
    0: 'Reduce consumption',
    1: 'Reduce consumption',
    2: 'Reduce consumption',
    3: 'Change transport',
    4: 'Recycle',
    5: 'Recycle'
}

answers_q107a_pollution_type = {
    0: 'High',
    1: 'Low',
    2: 'Low',
    3: 'Medium',
    4: 'Medium',
    5: 'Low',
}

answers_q107a_transport_type = {
    0: 'Private',
    1: 'Private',
    2: 'Private',
    3: 'Public',
    4: 'Public',
    5: 'Private',
}

In [18]:
mncn_df['Transport Pollution'] = mncn_df.apply(lambda row: answers_q107a_pollution_type[row['q107-A']], axis=1)
mncn_df['Transport Type'] = mncn_df.apply(lambda row: answers_q107a_transport_type[row['q107-A']], axis=1)

In [19]:
def reduce_consumption(q103):
    options = ['0', '1', '2']
    
    if not q103:
        return False
    else:
        if len(list(set(q103) & set(options))) > 0:
            return True
        else:
            return False
        
def change_transport(row):
    options = ['3']
    
    result = False
    
    if row['q103']:
        if len(list(set(row['q103']) & set(options))) > 0:
            result = True
    
    if row['q107-B'] == 1:
        result = True
        
    return result
        
def recycle(q103):
    options = ['4', '5']
    
    if not q103:
        return False
    else:
        if len(list(set(q103) & set(options))) > 0:
            return True
        else:
            return False

In [20]:
mncn_df['Reduce Consumption'] = mncn_df['q103'].apply(reduce_consumption)
mncn_df['Change Transport'] = mncn_df.apply(lambda row: change_transport(row), axis=1)
mncn_df['Recycle'] = mncn_df['q103'].apply(recycle)

In [21]:
mncn_df[['UserId', 'School', 'Grade', 'Type', 'Zone', 'Transport Pollution', 'Transport Type', 'Reduce Consumption', 'Change Transport', 'Recycle']].to_csv('../../data/MNCN/user_profiles_grouped.csv.csv')